In [1]:
import streamlit as st
import pandas as pd
from datetime import datetime
import shutil

# Load the product and price data from Excel files
@st.cache
def load_data(file_path):
    data = pd.read_excel(file_path)
    return data

product_data = load_data("products.xlsx")
price_data = load_data("price.xlsx")

# Function to update price data
def update_price_data(new_price, product_id):
    new_price_entry = {"Product ID": product_id, "Price": new_price, "Date": datetime.now()}
    price_data = load_data("price.xlsx")
    price_data = price_data.append(new_price_entry, ignore_index=True)
    price_data.to_excel("price.xlsx", index=False)

# Streamlit app
st.title("Restaurant POS System")

# Product catalog
st.header("Product Catalog")
st.dataframe(product_data)

# Shopping cart
st.header("Shopping Cart")

cart = st.session_state.cart if "cart" in st.session_state else []
total_price = sum([product_data.loc[product_data['Product ID'] == product_id, 'Price'].values[0] for product_id in cart])

for product_id in product_data['Product ID']:
    if st.button(f"Add to Cart (Product ID {product_id})"):
        cart.append(product_id)
        total_price += product_data.loc[product_data['Product ID'] == product_id, 'Price'].values[0]

st.write(f"Total Price: ${total_price:.2f}")

if st.button("Checkout"):
    transaction_time = datetime.now()
    transaction_data = {"Date/Time": transaction_time, "Total Cost": total_price, "Product Details": ", ".join([f"Product ID {product_id}" for product_id in cart])}
    
    # Save the transaction data to an Excel file
    transactions = load_data("transactions.xlsx") if "transactions" in st.session_state else pd.DataFrame()
    transactions = transactions.append(transaction_data, ignore_index=True)
    transactions.to_excel("transactions.xlsx", index=False)
    
    st.success("Checkout successful! Thank you for your purchase.")
    cart.clear()

# Developer Options
st.sidebar.header("Developer Options")

if st.sidebar.button("Update Price Data"):
    new_price = st.sidebar.number_input("New Price:", step=0.01)
    product_id = st.sidebar.number_input("Product ID:")
    update_price_data(new_price, product_id)
    st.sidebar.success("Price data updated.")

if st.sidebar.button("Clear Cart"):
    cart.clear()

if st.sidebar.button("Backup Data"):
    backup_folder = "backup_data"
    shutil.rmtree(backup_folder, ignore_errors=True)
    shutil.copy("products.xlsx", backup_folder)
    shutil.copy("price.xlsx", backup_folder)
    shutil.copy("transactions.xlsx", backup_folder)
    st.sidebar.success("Data backed up.")

if st.sidebar.button("Recover Data"):
    backup_folder = "backup_data"
    shutil.copy(f"{backup_folder}/products.xlsx", "products.xlsx")
    shutil.copy(f"{backup_folder}/price.xlsx", "price.xlsx")
    shutil.copy(f"{backup_folder}/transactions.xlsx", "transactions.xlsx")
    st.sidebar.success("Data recovered from backup.")

# Show Data
st.sidebar.header("Data Files")

if st.sidebar.button("Show Product Data"):
    st.sidebar.dataframe(product_data)

if st.sidebar.button("Show Price Data"):
    st.sidebar.dataframe(price_data)

if st.sidebar.button("Show Transaction Data"):
    transaction_data = load_data("transactions.xlsx")
    st.sidebar.dataframe(transaction_data)

2023-11-01 09:23:35.416 
  command:

    streamlit run C:\Users\layto\anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


InternalHashError: module '__main__' has no attribute '__file__'

While caching the body of `load_data()`, Streamlit encountered an
object of type `builtins.function`, which it does not know how to hash.

**In this specific case, it's very likely you found a Streamlit bug so please
[file a bug report here.]
(https://github.com/streamlit/streamlit/issues/new/choose)**

In the meantime, you can try bypassing this error by registering a custom
hash function via the `hash_funcs` keyword in @st.cache(). For example:

```
@st.cache(hash_funcs={builtins.function: my_hash_func})
def my_func(...):
    ...
```

If you don't know where the object of type `builtins.function` is coming
from, try looking at the hash chain below for an object that you do recognize,
then pass that to `hash_funcs` instead:

```
Object of type builtins.function: <function load_data at 0x000001F527D97430>
```

Please see the `hash_funcs` [documentation](https://docs.streamlit.io/library/advanced-features/caching#the-hash